In [94]:
import pandas as pd
import matplotlib.pyplot as plt
import math
import datetime
import numpy as np

## Reading excel file


In [95]:
xls = pd.ExcelFile('data.xlsx')

KeyboardInterrupt: 

## Reading individual Sheets

In [96]:
# Industrialisation DataFrame
ind = pd.read_excel(xls, 'industrialisation')
ind.head()
ind.shape

(2840, 30)

In [97]:
# CO2 dataframe
co2 = pd.read_excel(xls, 'Co2')
co2

,Entity,Code,Year,Annual CO2 emissions (tonnes )
0,Afghanistan,AFG,1949,14656.00
1,Afghanistan,AFG,1950,84272.00
2,Afghanistan,AFG,1951,91600.00
3,Afghanistan,AFG,1952,91600.00
4,Afghanistan,AFG,1953,106256.00
...,...,...,...,...
20848,Zimbabwe,ZWE,2013,11536239.29
20849,Zimbabwe,ZWE,2014,11866348.41
20850,Zimbabwe,ZWE,2015,10907603.94
20851,Zimbabwe,ZWE,2016,9932649.88


In [21]:
co2.shape

(20853, 4)

In [120]:
# Global Temps Dataframe
gltemp = pd.read_excel(xls, 'GlobalLandTemperaturesByCity')
gltemp.head()
gltemp.shape

(1048575, 7)

## Cleaning the DFs

### Cleaning Global Temps By City

In [121]:
gltemp.isna().sum()

dt                                   0
AverageTemperature               47547
AverageTemperatureUncertainty    47547
City                                 0
Country                              0
Latitude                             0
Longitude                            0
dtype: int64

In [122]:
gltemp.iloc[798583]['dt']

datetime.datetime(1900, 7, 1, 0, 0)

In [123]:
gltemp = gltemp.dropna()

In [124]:
gltemp.isna().sum()

dt                               0
AverageTemperature               0
AverageTemperatureUncertainty    0
City                             0
Country                          0
Latitude                         0
Longitude                        0
dtype: int64

In [125]:
print(gltemp.iloc[798580])

dt                               1987-07-01 00:00:00
AverageTemperature                             22.97
AverageTemperatureUncertainty                  0.207
City                                           Benxi
Country                                        China
Latitude                                      40.99N
Longitude                                    123.55E
Name: 836224, dtype: object


In [126]:
gltemp = gltemp.reset_index()

In [127]:
del gltemp['index']

In [128]:
gltemp.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1743-11-01,6.068,1.737,Ã…rhus,Denmark,57.05N,10.33E
1,1744-04-01,5.788,3.624,Ã…rhus,Denmark,57.05N,10.33E
2,1744-05-01,10.644,1.283,Ã…rhus,Denmark,57.05N,10.33E
3,1744-06-01,14.051,1.347,Ã…rhus,Denmark,57.05N,10.33E
4,1744-07-01,16.082,1.396,Ã…rhus,Denmark,57.05N,10.33E


In [129]:
gltemp['dt'] = pd.to_datetime(gltemp['dt'])

In [130]:
gltemp.head()


,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1743-11-01,6.068,1.737,Ã…rhus,Denmark,57.05N,10.33E
1,1744-04-01,5.788,3.624,Ã…rhus,Denmark,57.05N,10.33E
2,1744-05-01,10.644,1.283,Ã…rhus,Denmark,57.05N,10.33E
3,1744-06-01,14.051,1.347,Ã…rhus,Denmark,57.05N,10.33E
4,1744-07-01,16.082,1.396,Ã…rhus,Denmark,57.05N,10.33E


In [131]:
gltemp = gltemp[gltemp['dt'] > pd.Timestamp(year=1989, day=31, month=12)]

In [132]:
gltemp.head()
gltemp.shape

(119879, 7)

In [133]:
gltemp = gltemp.reset_index(drop=True)

In [134]:
gltemp

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1990-01-01,4.267,0.459,Ã…rhus,Denmark,57.05N,10.33E
1,1990-02-01,5.895,0.460,Ã…rhus,Denmark,57.05N,10.33E
2,1990-03-01,6.565,0.299,Ã…rhus,Denmark,57.05N,10.33E
3,1990-04-01,8.068,0.222,Ã…rhus,Denmark,57.05N,10.33E
4,1990-05-01,12.944,0.287,Ã…rhus,Denmark,57.05N,10.33E
...,...,...,...,...,...,...,...
119874,2013-04-01,27.745,0.241,Bontang,Indonesia,0.80N,118.13E
119875,2013-05-01,27.795,0.258,Bontang,Indonesia,0.80N,118.13E
119876,2013-06-01,28.220,0.292,Bontang,Indonesia,0.80N,118.13E
119877,2013-07-01,27.127,0.231,Bontang,Indonesia,0.80N,118.13E


In [135]:
one = gltemp.groupby('City').aggregate(np.mean)
one

,AverageTemperature,AverageTemperatureUncertainty
City,,
A CoruÃ±a,14.038408,0.371757
Aachen,9.889461,0.220574
Aalborg,8.786412,0.283239
Aba,27.216588,0.470606
Abadan,26.111243,0.518232
...,...,...
ÃœrÃ¼mqi,7.153908,0.588574
Ã–skemen,2.635722,0.478335
Ã‡orlu,14.128972,0.310257
